# Evaluating Preference Independence using Regression Trees

From prior analysis, we have reason to believe that the causal impact of different conditions in a conjoint experiment on support for business development investment may not be independent. In particular, there is evidence that when one varies the potential economic returns to the investment, one sees very different patterns in the causal effects of different information treatments related to the environmental impact of these projects. 

To illustrate, consider the following two plots. In the first, we show how the causal effect of different job-creation treatments does not vary with environmental certification treatments:

![no_dependency](40_docs/images/no_dependency.png)![legend](40_docs/images/legend.png)

As the figure shows, regardess of whether the investment is said to be pursuing or is said to have received environmental certification (i.e. across the three certification treatments), the marginal impact of moving from 100 jobs to 1000 jobs is constant, and the marginal impact of moving from 1,000 to 100,000 jobs is constant. 

My contrast, if we look at the marginal effect of different waste treatment conditions, the marginal effects vary radically. 

![dependent_prefs](40_docs/images/dependent_prefs.png)![legend](40_docs/images/legend.png)

If waste is low (100HH), then moving from 100 jobs to 1,000 jobs has about the same marginal effect as moving from 1,000 to 100,000. If, by contrast, there is waste 300HH of waste, then the marginal effect of moving from 100 to 1,000 jobs is large, but there's no effect of moving from 1,000 to 100,000. And if there is 1,000 HH of waste, then moving from 100 to 1,000 jobs has no effect, but moving from 1,000 to 100,000 jobs has a large effect. 

This is evidence of interdependence of effects. In addition (though this is not in the paper), this interdependence appears to be asymmetric: while the waste impact of a project impacts the effect of different job sizes, it is not clear the reverse is true (**right? Can't quite remember what slide you showed me!**)


## Load and Clean Data

In [ ]:
import os 
import pandas as pd
os.chdir('/users/nick/github/preference_interdependence')
df = pd.read_stata('00_source_data/Fish_VN2018_rep.dta')

In [ ]:
df.head()

In [ ]:
# Just lookin' around...
df.sample(5).transpose()

In [ ]:
# Subset to core treatments
df = df[df.columns[3:12].to_list() + ['choice']]

In [ ]:
df.head()

In [ ]:
# Convert from a heterogeneous table to a homogenous array 
# (scikit-learn hates heterogeneous tables)

# Build formula for converting categoricals 
# to one-hot vectors
import patsy
formula = ''
for i in df.columns[:-1]:
    formula = formula + f'C({i}) + '
formula[:-2]

In [ ]:
# Move categoricals to one-hot encodings (i.e. indicator variables)
df_y , df_X = patsy.dmatrices('choice ~ ' + formula[:-2], df, return_type='dataframe')

In [ ]:
# Split the data for training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_X.values, df_y.values, 
                                                    test_size=0.5, 
                                                    train_size=0.5,
                                                    random_state=42)

In [ ]:
from sklearn import tree

my_model = tree.DecisionTreeClassifier(max_depth=3)
my_model.fit(X_train, y_train)
#tree.plot_tree(my_model)

In [ ]:
r = tree.export_text(my_model, feature_names=df_X.columns.tolist())
print(r)

In [ ]:
import graphviz
dot_data = tree.export_graphviz(my_model, out_file=None, 
                                feature_names=df_X.columns.tolist())
graph = graphviz.Source(dot_data)  
graph 